In [1]:
from mplsoccer import Sblocal
import pandas as pd
import os
import re
import glob
import numpy as np
from tqdm import tqdm

In [2]:
parser = Sblocal()

# Collecting Matches data (Statsbomb)

In [3]:
#from Matches folder fetching all the Competitions(CompID) played
#from each folder within Matches categorised on CompetitionID
#fetching the list of SeasonsID(seasons played data) under that competition

dfMatcheslist = []
MATCHES = 'open-data-master/data/matches'
matches_compID_wise = os.listdir(MATCHES)
for i in enumerate(matches_compID_wise):
    if re.findall("\D",i[1]):
        remove = i[1]
        matches_compID_wise.remove(remove)
    else:
        #print(os.path.join(MATCHES,i[1]))
        MATCHES_compid = os.path.join(MATCHES,i[1])
        for j in os.listdir(MATCHES_compid):
            #print(i[1]," -> ",j)
            MATCHES_compid_sznid = os.path.join(MATCHES_compid,j)
            dfMatcheslist.append(parser.match(MATCHES_compid_sznid))
for i in enumerate(dfMatcheslist):
     if type(i[1])!=pd.DataFrame:
            del dfMatcheslist[i[0]]
dfMatches = pd.concat(dfMatcheslist)


# Collecting Events and Lineup data (Statsbomb)

In [4]:
def parse_event_lineup_data(name):
    if name == 'events':
        dfEventslist = []
        dfRelatedlist = []
        dfFreezelist = []
        dfTacticslist = []
    else:
        dfLineupslist = []
    
    general_path = 'open-data-master/data'
    data_path = os.path.join(general_path,name)
    if name=='events':
        EVENTS_matchid = os.listdir(data_path)
        for i in tqdm(EVENTS_matchid):
            event_path = os.path.join(data_path,i)
            event,related,freeze,tactics = parser.event(event_path)
            dfEventslist.append(event)
            dfRelatedlist.append(related)
            dfFreezelist.append(freeze)
            dfTacticslist.append(tactics)
            
    else:
        LINEUPS_matchid = os.listdir(data_path)
        for i in tqdm(LINEUPS_matchid):
            lineup_path = os.path.join(data_path,i)
            lineup = parser.event(lineup_path)
            dfLineupslist.append(lineup)
        
    if name=='events':
        return (dfEventslist,dfRelatedlist,dfFreezelist,
                   dfTacticslist)
    else:
        return dfLineupslist

In [5]:
events,related,freeze,tactics = parse_event_lineup_data('events')

100%|███████████████████████████████████████| 1321/1321 [14:08<00:00,  1.56it/s]


In [111]:
!pip install databricks

In [113]:
!pip install koalas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.8/390.8 kB 2.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 2.1 MB/s eta 0:00:0000:0100:01


In [115]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 2.1 MB/s eta 0:00:0000:0100:03
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 2.2 MB/s eta 0:00:00a 0:00:01
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824009 sha256=35d729f8f47767b99fb9dbbeccca1edaf269ac48b0ca8659b8bded07e69d5206
  Stored in directory: /Users/prabalsingh/Library/Caches/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [6]:
events[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3561 entries, 0 to 3560
Data columns (total 70 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              3561 non-null   object 
 1   index                           3561 non-null   int64  
 2   period                          3561 non-null   int64  
 3   timestamp                       3561 non-null   object 
 4   minute                          3561 non-null   int64  
 5   second                          3561 non-null   int64  
 6   possession                      3561 non-null   int64  
 7   duration                        2740 non-null   float64
 8   match_id                        3561 non-null   int64  
 9   type_id                         3561 non-null   int64  
 10  type_name                       3561 non-null   object 
 11  possession_team_id              3561 non-null   int64  
 12  possession_team_name            35

<b>Fetching x,y,end_x,end_y</b>

In [7]:
from mplsoccer import Standardizer
standard = Standardizer(pitch_from='statsbomb', pitch_to='uefa')
for i in tqdm(events):
    new_x, new_y = standard.transform(i.x, i.y)
    i.x = new_x
    i.y = new_y
    new_end_x, new_end_y = standard.transform(i.end_x, i.end_y)
    i.end_x = new_end_x
    i.end_y = new_end_y

100%|██████████████████████████████████████| 1321/1321 [00:05<00:00, 252.73it/s]


<b>Fetching goalkeeper_x,goalkeeper_y for shots</b>

In [8]:
# creating a dataframe of goalkeeper position for shots

# gk = related[0][(related[0].type_name_related=='Goal Keeper') & 
#            (related[0].type_name=='Shot')][['id', 'id_related']]
# gk.merge((events[0][['x','y','id', 'player_id', 'type_name', 'sub_type_name']]
#                      .rename({'id': 'id_related'}, axis=1)), how='left', on='id_related')

df_gk_list = []
for i,j in tqdm(zip(related,events)):
    df_gk = i[(i.type_name_related=='Goal Keeper') & 
                       (i.type_name=='Shot')][['id', 'id_related']]
    df_gk_list.append(df_gk.merge((j[['x','y','id', 'player_id', 'type_name', 'sub_type_name']]
                 .rename({'id': 'id_related'}, axis=1)), how='left', on='id_related'))

1321it [00:18, 69.79it/s]


<b>Fetching carry_length</b>

In [9]:
for i in tqdm(events):
    i['carry_length'] = (((i.x - i.end_x))** 2 +
                         ((i.y - i.end_y)) ** 2) ** 0.5

100%|██████████████████████████████████████| 1321/1321 [00:01<00:00, 704.17it/s]


<b>Standardizing body_part_name (left foot, right foot, other)</b>

In [10]:
for i in tqdm(events):
    i['body_part_name'] = i.body_part_name.replace({'Head': 'Other'})

100%|██████████████████████████████████████| 1321/1321 [00:09<00:00, 135.37it/s]


<b>Strong Foot</b>

In [11]:
df_foot_list = []
for i in tqdm(events):
    df_foot_list.append(i[['player_id','body_part_name']])
df_foot = pd.concat(df_foot_list)
df_foot.shape

100%|███████████████████████████████████████| 1321/1321 [00:13<00:00, 97.36it/s]


(4665632, 2)

In [12]:
df_foot = df_foot[df_foot.body_part_name.isin(['Right Foot', 'Left Foot'])]
df_foot = df_foot.groupby('player_id').body_part_name.value_counts()
df_foot = df_foot.unstack()
df_foot.reset_index(drop=False, inplace=True)
df_foot.index.name = ''
df_foot.rename({'Left Foot': 'left', 'Right Foot': 'right'}, axis=1, inplace=True)
df_foot.fillna(0, inplace=True)
n_total = df_foot[['left', 'right']].sum(axis=1)
df_foot['left'] = df_foot['left'] / n_total
df_foot['right'] = df_foot['right'] / n_total
df_foot.loc[(np.abs(df_foot.left - df_foot.right) < 0.1) & n_total > 20, 'foot'] = 'both'
df_foot.loc[(df_foot.foot.isnull()) & (df_foot.left > df_foot.right), 'foot'] = 'left'
df_foot.loc[(df_foot.foot.isnull()), 'foot'] = 'right'
#df_foot.shape

In [13]:
#events[69].merge(df_foot[['player_id','foot']],on='player_id').rename(columns={'foot':'strong_foot'})
for i in tqdm(range(len(events))):
    events[i] = events[i].merge(df_foot[['player_id','foot']],on='player_id').rename(
        columns={'foot':'strong_foot'}) 

100%|███████████████████████████████████████| 1321/1321 [00:44<00:00, 29.41it/s]


<b>Fetching fast_break</b>

In [14]:
"""fast_break - whether the team wins the ball in own third
and shoots in the last quarter of the pitch within 25 seconds"""

for i in tqdm(range(len(events))):
    events[i]['eventSec'] = events[i].timestamp.apply(lambda x: x.minute * 60 + x.second + x.microsecond / 1000000)
mask_gk = ((events[i].type_name == 'Goal Keeper') & 
           (~events[i].sub_type_name.isin(['Shot Faced', 'Goal Conceded', 'Penalty Conceded'])))
mask_50 =((events[i].type_name == '50/50') & 
          (events[i].outcome_name.isin(['Success To Team', 'Won','Success To Opposition'])))
mask_other = (events[i].type_name == 'Interception') | (events[i].sub_type_name == 'Tackle')
mask_defence_win = (mask_gk | mask_50 | mask_other) & (events[i].x < 35)
events[i].loc[mask_defence_win, 'defence_win'] = events[i].loc[mask_defence_win, 'team_id']
events[i].loc[mask_defence_win, 'defence_sec'] = events[i].loc[mask_defence_win, 'eventSec']
group_match = events[i].groupby(['match_id', 'period'])
events[i][['defence_win', 'defence_sec']] = group_match[['defence_win', 'defence_sec']].ffill()
mask_fast = (((events[i].eventSec - events[i].defence_sec) <= 25) &
             (events[i].x > 78.75) &
             (events[i].type_name == 'Shot') & (events[i].team_id == events[i].defence_win))
events[i]['fast_break'] = mask_fast

100%|██████████████████████████████████████| 1321/1321 [00:06<00:00, 191.45it/s]


<b> Fetching shot_type_name </b>

In [15]:
for i in tqdm(range(len(events))):
    mask_set_piece = events[i].sub_type_name.isin(['Throw-in', 'Corner', 'Free Kick'])
    events[i].loc[mask_set_piece, 'set_piece'] = events[i].loc[mask_set_piece, 'sub_type_name']

100%|██████████████████████████████████████| 1321/1321 [00:07<00:00, 171.43it/s]


In [16]:
""" Flag 10 seconds from a corner or freekick/ 20 seconds from a throw-in """

for i in tqdm(range(len(events))):
    group_match = events[i].groupby(['match_id', 'period'])
    for set_piece in ['Corner', 'Throw-in', 'Free Kick']:
        mask = events[i].set_piece == set_piece
        name = set_piece.replace(' ', '_').replace('-', '_').lower()
        events[i].loc[mask, f'{name}_sec'] = events[i].loc[mask, 'eventSec']
        events[i].loc[mask, f'{name}_team'] = events[i].loc[mask, 'team_id']
        events[i][f'{name}_sec'] = group_match[f'{name}_sec'].ffill()
        events[i][f'{name}_team'] = group_match[f'{name}_team'].ffill()
        events[i][f'{name}_sec'] = events[i].eventSec - events[i][f'{name}_sec']
    events[i].loc[events[i].throw_in_sec > 20, 'throw_in_sec'] = np.nan
    events[i].loc[events[i].free_kick_sec > 10, 'free_kick_sec'] = np.nan
    events[i].loc[events[i].corner_sec > 10, 'corner_sec'] = np.nan
    events[i]['play_type'] = events[i][['throw_in_sec', 'free_kick_sec', 'corner_sec']].idxmin(axis=1).str[:-4]
    print(events[i][['throw_in_sec', 'free_kick_sec', 'corner_sec']].idxmin(axis=1).str[:-4])
    # if throw-in and defensive set to null
    mask_defensive = ((events[i].play_type == 'throw_in') &
                      (events[i]['throw_in_team'] != events[i].team_id))
    events[i].loc[mask_defensive, 'play_type'] = np.nan

  0%|▏                                         | 5/1321 [00:00<01:09, 18.89it/s]

0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
         ...  
3545    corner
3546    corner
3547    corner
3548    corner
3549    corner
Length: 3550, dtype: object
0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
          ...   
3006         NaN
3007         NaN
3008         NaN
3009    throw_in
3010         NaN
Length: 3011, dtype: object
0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
3751    NaN
3752    NaN
3753    NaN
3754    NaN
3755    NaN
Length: 3756, dtype: object
0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
3345    NaN
3346    NaN
3347    NaN
3348    NaN
3349    NaN
Length: 3350, dtype: object
0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
          ...    
3519          NaN
3520          NaN
3521          NaN
3522          NaN
3523    free_kick
Length: 3524, dtype: object


  1%|▎                                         | 8/1321 [00:00<01:03, 20.81it/s]

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
3414    NaN
3415    NaN
3416    NaN
3417    NaN
3418    NaN
Length: 3419, dtype: object
0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
3274    NaN
3275    NaN
3276    NaN
3277    NaN
3278    NaN
Length: 3279, dtype: object
0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
          ...   
2703    throw_in
2704    throw_in
2705    throw_in
2706    throw_in
2707    throw_in
Length: 2708, dtype: object
0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
3812    NaN
3813    NaN
3814    NaN
3815    NaN
3816    NaN
Length: 3817, dtype: object


  1%|▍                                        | 14/1321 [00:00<01:03, 20.45it/s]

...

0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
         ...  
3724    corner
3725    corner
3726    corner
3727    corner
3728    corner
Length: 3729, dtype: object
0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
          ...   
2853    throw_in
2854    throw_in
2855    throw_in
2856    throw_in
2857    throw_in
Length: 2858, dtype: object
0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
          ...   
4213    throw_in
4214    throw_in
4215    throw_in
4216    throw_in
4217    throw_in
Length: 4218, dtype: object
0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
4009    NaN
4010    NaN
4011    NaN
4012    NaN
4013    NaN
Length: 4014, dtype: object
0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
          ...   
3954    throw_in
3955    throw_in
3956         NaN
3957         NaN
3958         NaN
Length: 3959, dtype: object


100%|██████████████████████████████████████▊| 1315/1321 [00:47<00:00, 24.28it/s]

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
3712    NaN
3713    NaN
3714    NaN
3715    NaN
3716    NaN
Length: 3717, dtype: object
0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
3739    NaN
3740    NaN
3741    NaN
3742    NaN
3743    NaN
Length: 3744, dtype: object
0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
3938    NaN
3939    NaN
3940    NaN
3941    NaN
3942    NaN
Length: 3943, dtype: object
0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
         ...  
4660    corner
4661    corner
4662    corner
4663    corner
4664    corner
Length: 4665, dtype: object
0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
          ...    
3509    free_kick
3510    free_kick
3511    free_kick
3512    free_kick
3513    free_kick
Length: 3514, dtype: object
0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
4370    NaN
4371    NaN
4372 

100%|███████████████████████████████████████| 1321/1321 [00:47<00:00, 27.94it/s]

0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
         ...  
2872    corner
2873    corner
2874    corner
2875    corner
2876    corner
Length: 2877, dtype: object
0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
4098    NaN
4099    NaN
4100    NaN
4101    NaN
4102    NaN
Length: 4103, dtype: object
0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
          ...    
3386    free_kick
3387    free_kick
3388    free_kick
3389    free_kick
3390          NaN
Length: 3391, dtype: object
0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
3671    NaN
3672    NaN
3673    NaN
3674    NaN
3675    NaN
Length: 3676, dtype: object
0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
          ...    
3128    free_kick
3129    free_kick
3130    free_kick
3131    free_kick
3132    free_kick
Length: 3133, dtype: object
0       NaN
1       NaN
2       NaN

In [17]:
# Filtering all shots
dfShots_statsbomb_list = []
shotsCount = 0
goalsCount = 0
for i in tqdm(range(len(events))):
    temp = (events[i][(events[i].type_name=='Shot')&
                                            (~events[i].sub_type_name.isin(['Penalty', 'Corner']))]
                          .dropna(how='all', axis=1)
                          .copy())
    dfShots_statsbomb_list.append(temp)
    shotsCount = shotsCount + temp.shape[0]
    goalsCount = goalsCount + temp[temp.outcome_name=='Goal'].shape[0]
print("Shots attempted in the Statsbomb data = ",shotsCount)
print("Goals scored in the Statsbomb data = ",goalsCount)

100%|███████████████████████████████████████| 1321/1321 [00:19<00:00, 68.55it/s]

Shots attempted in the Statsbomb data =  33045
Goals scored in the Statsbomb data =  3710


In [18]:
# shot_type_name - {open_play, throw_in(25sec), direct_set_piece(free kick,kick off)}
# excludes corner, penalties

for i in tqdm(range(len(dfShots_statsbomb_list))):
    dfShots_statsbomb_list[i].sub_type_name.replace({'Open Play': np.nan,
                                              'Free Kick': 'direct_set_piece',
                                              'Kick Off': 'direct_set_piece'}, inplace=True)
    mask_null = dfShots_statsbomb_list[i].sub_type_name.isnull()
    dfShots_statsbomb_list[i].loc[mask_null, 'sub_type_name'] = dfShots_statsbomb_list[i].loc[mask_null, 'play_type']
    dfShots_statsbomb_list[i].loc[dfShots_statsbomb_list[i].sub_type_name.isnull(), 'sub_type_name'] = 'open_play'
    dfShots_statsbomb_list[i].rename({'sub_type_name': 'shot_type_name'}, axis=1, inplace=True)

100%|██████████████████████████████████████| 1321/1321 [00:02<00:00, 549.21it/s]


<b> Fetching player_name </b>

In [19]:
for i in tqdm(range(len(events))):
    player_name_series = events[i].player_name.str.strip().str.replace(pat='"', repl="'").str.split(' ')
    events[i]['firstName'] = player_name_series.apply(lambda x: x[0] if isinstance(x, list) else None)
    events[i]['middleName'] = player_name_series.apply(lambda x: ' '.join(x[1:-1]) if isinstance(x, list) else None)
    events[i]['middleName'] = events[i]['middleName'].str.strip()
    events[i]['lastName'] = player_name_series.apply(lambda x: x[-1] if isinstance(x, list) else None)
    events[i]['Name'] = ((events[i]['firstName'] + ' ' + events[i]['middleName']).str.strip()
                                  + ' ' + events[i]['lastName'])

100%|███████████████████████████████████████| 1321/1321 [00:39<00:00, 33.10it/s]


<b>Competition gender</b>

In [20]:
for i in tqdm(range(len(dfShots_statsbomb_list))):
    dfShots_statsbomb_list[i] = dfShots_statsbomb_list[i].merge(
        dfMatches[['match_id','home_team_gender']].rename(
            {'home_team_gender': 'competition_gender'},axis='columns'),
        on='match_id', how='left')

100%|██████████████████████████████████████| 1321/1321 [00:07<00:00, 182.99it/s]


<b>pass-assist information</b>

In [21]:
for i in tqdm(range(len(events))):
    pass_cols = ['pass_assisted_shot_id', 'end_x', 'end_y', 'pass_height_name',
                 'pass_switch', 'technique_name', 'pass_cross', 'pass_cut_back']
    # masking the required pass_assist rows
    mask_pass_assist = events[i].pass_assisted_shot_id.notnull()
    # creating anfy missing element of pass_cols if not present in events df
    for j in pass_cols:
        if j not in events[i].columns:
            #print(j," - to create")
            events[i][j] = np.nan
    
    df_pass = events[i].loc[mask_pass_assist, pass_cols]
    df_pass.rename({'pass_assisted_shot_id': 'id', 'end_x': 'pass_end_x', 'end_y': 'pass_end_y'}, axis=1, inplace=True)
    df_pass['pass_switch'].fillna(False, inplace=True)
    df_pass['pass_cross'].fillna(False, inplace=True)
    df_pass['pass_cut_back'].fillna(False, inplace=True)
    dfShots_statsbomb_list[i] = dfShots_statsbomb_list[i].merge(df_pass, on='id', how='left', suffixes=['', '_pass'])
    dfShots_statsbomb_list[i].rename({'technique_name_pass': 'pass_technique_name'}, axis=1, inplace=True)

100%|███████████████████████████████████████| 1321/1321 [00:26<00:00, 48.95it/s]


In [22]:
for j in range(3):
    idx_to_del = []
    cnt = 0
    for i in enumerate(dfShots_statsbomb_list[j].columns):
        if (i[1].split("_")[-1]=='pass'):
            idx_to_del.append(i[0])
        elif (i[1] == 'pass_technique_name'):
            cnt += 1
            if cnt>1:
                idx_to_del.append(i[0])
    dfShots_statsbomb_list[j].drop(
        dfShots_statsbomb_list[j].columns[idx_to_del],
        axis=1,inplace=True)

In [23]:
for i in range(3):
    mask_pass_assist = events[i].pass_assisted_shot_id.notnull()
    df_pass = events[i].loc[mask_pass_assist,
                            ['pass_assisted_shot_id','technique_name']]
    df_pass.rename({'pass_assisted_shot_id': 'id'}, axis=1, inplace=True)
    dfShots_statsbomb_list[i] = dfShots_statsbomb_list[i].merge(df_pass, on='id', how='left', suffixes=['', '_pass'])
    dfShots_statsbomb_list[i].rename({'technique_name_pass': 'pass_technique_name'}, axis=1, inplace=True)

<b>simplifying pass_height</b>

In [24]:
for i in tqdm(range(len(dfShots_statsbomb_list))):
    dfShots_statsbomb_list[i].pass_height_name.replace(
        {'Ground Pass': 'Ground/ Low Pass',
        'Low Pass': 'Ground/ Low Pass'}, inplace=True)

  0%|                                                  | 0/1321 [00:00<?, ?it/s]/var/folders/b5/p9rmdc817y36m0hgtkqmx93m0000gn/T/ipykernel_89128/1564765512.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfShots_statsbomb_list[i].pass_height_name.replace(
100%|██████████████████████████████████████| 1321/1321 [00:03<00:00, 429.69it/s]


<b>Fetching different Angles and Distances for the shot</b>

In [25]:
from mplsoccer.dimensions import create_pitch_dims
dim_uefa = create_pitch_dims(pitch_type='uefa')
goal_width = dim_uefa.goal_width

for i in tqdm(range(len(dfShots_statsbomb_list))):
    dx = abs(dim_uefa.right - dfShots_statsbomb_list[i].x)
    dy = abs(dim_uefa.center_width - dfShots_statsbomb_list[i].y)
    dfShots_statsbomb_list[i]['visible_angle'] = np.arctan2(goal_width * dx , (dx**2 + dy**2 - (goal_width / 2.) ** 2))
    dfShots_statsbomb_list[i]['middle_angle'] = np.arctan2(dy, dx)
    dfShots_statsbomb_list[i]['distance_to_goal'] = round((dy**2 + dx**2)**0.5, 1)

100%|██████████████████████████████████████| 1321/1321 [00:02<00:00, 481.98it/s]


In [26]:
# interaction b/w angle and distance for the shot
for i in tqdm(range(len(dfShots_statsbomb_list))):
    dfShots_statsbomb_list[i]['distance_visible_angle_interaction'] = dfShots_statsbomb_list[i].distance_to_goal * dfShots_statsbomb_list[i].visible_angle

100%|██████████████████████████████████████| 1321/1321 [00:02<00:00, 440.96it/s]


In [27]:
# logarithmic distance [for distribution skewness]
for i in tqdm(range(len(dfShots_statsbomb_list))):
    dfShots_statsbomb_list[i]['log_distance'] = np.log(
        dfShots_statsbomb_list[i].distance_to_goal)

100%|█████████████████████████████████████| 1321/1321 [00:00<00:00, 2251.32it/s]


<b> Fetching assist_type</b>

In [28]:
for i in tqdm(range(len(events))):
    # first filter out some events so the previous event is the correct assist type
    on_ball = ['Ball Recovery','Block','Clearance','Dispossessed','Dribble','Referee Ball-Drop','Shield',
               'Error','Foul Won','Goal Keeper','Interception','Miscontrol','Own Goal Against','Pass','Shot']
    # won 50/50s
    mask_50_50_success = ((events[i].type_name=='50/50')&
                          (events[i].outcome_name.isin(['Won','Success To Team','Success To Opposition'])))
    # off ball goal keeper
    mask_goalkeeper_off_ball = events[i].sub_type_name.isin(['Shot Faced', 'Goal Conceded',
                                                                      'Penalty Conceded', 'Smother'])
    # all on the ball events
    mask_on_ball = ((events[i].type_name.isin(on_ball))&(~mask_goalkeeper_off_ball))|(mask_50_50_success)
    events[i] = events[i][mask_on_ball].copy()
    match_group = events[i].groupby(['match_id', 'period'])
    for j in range(1, 4):
        events[i][f'prev_id_{j}'] = match_group.id.shift(j)
        events[i][f'prev_type_name_{j}'] = match_group.type_name.shift(j)
        events[i][f'prev_player_id_{j}'] = match_group.player_id.shift(j)
        events[i][f'prev_team_id_{j}'] = match_group.team_id.shift(j)

100%|███████████████████████████████████████| 1321/1321 [00:41<00:00, 32.08it/s]


In [29]:
for i in tqdm(range(len(events))):
    dfShots_statsbomb_list[i] = (dfShots_statsbomb_list[i].merge(events[i][[
        'prev_id_1','prev_type_name_1','prev_player_id_1','prev_team_id_1',
    'prev_id_2','prev_type_name_2','prev_player_id_2','prev_team_id_2',
    'prev_id_3','prev_type_name_3','prev_player_id_3','prev_team_id_3','id']],
        on='id'))

100%|███████████████████████████████████████| 1321/1321 [00:28<00:00, 45.73it/s]


In [30]:
for i in tqdm(range(len(dfShots_statsbomb_list))):
    dfShots_statsbomb_list[i].loc[
        dfShots_statsbomb_list[i].pass_end_x.notnull(), 'assist_type'] = 'pass'

    dfShots_statsbomb_list[i].loc[
        dfShots_statsbomb_list[i].shot_type_name.isin(
            ['Free Kick', 'Corner', 'Kick Off']), 'assist_type'] = 'direct'

    dfShots_statsbomb_list[i].loc[
        dfShots_statsbomb_list[i].assist_type.isnull() &
                           ((dfShots_statsbomb_list[i].prev_type_name_3.isin(
                               ['Shot', 'Goal Keeper'])) |
                            (dfShots_statsbomb_list[i].prev_type_name_2.isin(
                                ['Shot', 'Goal Keeper'])) |
                            (dfShots_statsbomb_list[i].prev_type_name_1.isin(
                                ['Shot', 'Goal Keeper']))), 'assist_type'] = 'rebound'

    dfShots_statsbomb_list[i].loc[
        dfShots_statsbomb_list[i].assist_type.isnull() &
                           ((dfShots_statsbomb_list[i].prev_type_name_3.isin(
                               ['Clearance'])) |
                            (dfShots_statsbomb_list[i].prev_type_name_2.isin(
                                ['Clearance'])) |
                            (dfShots_statsbomb_list[i].prev_type_name_1.isin(
                                    ['Clearance']))), 'assist_type'] = 'clearance'

    dfShots_statsbomb_list[i].loc[
        dfShots_statsbomb_list[i].assist_type.isnull(), 'assist_type'] = 'recovery'

100%|██████████████████████████████████████| 1321/1321 [00:08<00:00, 154.09it/s]


<b>Adding Target Coloumn.</b> 
    

In [31]:
for i in tqdm(range(len(dfShots_statsbomb_list))):
    dfShots_statsbomb_list[i]['goal'] = dfShots_statsbomb_list[i].outcome_name == 'Goal'

100%|██████████████████████████████████████| 1321/1321 [00:01<00:00, 815.61it/s]


<b>Transforming some columns into boolean</b>

In [32]:
# creating col shot_open_goal where ever missing in shot dataframe list
for i in tqdm(range(len(dfShots_statsbomb_list))):
    if "shot_open_goal" not in dfShots_statsbomb_list[i].columns:
        dfShots_statsbomb_list[i]['shot_open_goal'] = np.nan

100%|█████████████████████████████████████| 1321/1321 [00:00<00:00, 2851.39it/s]


In [33]:
# creating col under_pressure where ever missing in shot dataframe list
for i in tqdm(range(len(dfShots_statsbomb_list))):
    if "under_pressure" not in dfShots_statsbomb_list[i].columns:
        dfShots_statsbomb_list[i]['under_pressure'] = np.nan

100%|████████████████████████████████████| 1321/1321 [00:00<00:00, 16145.43it/s]


In [34]:
# creating col shot_one_on_one where ever missing in shot dataframe list
for i in tqdm(range(len(dfShots_statsbomb_list))):
    if "shot_one_on_one" not in dfShots_statsbomb_list[i].columns:
        dfShots_statsbomb_list[i]['shot_one_on_one'] = np.nan

100%|████████████████████████████████████| 1321/1321 [00:00<00:00, 16585.67it/s]


In [35]:
for i in tqdm(range(len(dfShots_statsbomb_list))):    
    dfShots_statsbomb_list[i]['shot_open_goal'].fillna(False, inplace=True)
    dfShots_statsbomb_list[i]['under_pressure'] = dfShots_statsbomb_list[i]['under_pressure'].fillna(0).astype(bool)
    dfShots_statsbomb_list[i]['shot_one_on_one'].fillna(False, inplace=True)

  0%|                                                  | 0/1321 [00:00<?, ?it/s]/var/folders/b5/p9rmdc817y36m0hgtkqmx93m0000gn/T/ipykernel_89128/961496028.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfShots_statsbomb_list[i]['shot_open_goal'].fillna(False, inplace=True)
/var/folders/b5/p9rmdc817y36m0hgtkqmx93m0000gn/T/ipykernel_89128/961496028.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfShots_statsbomb_list[i]['shot_one_on_one'].fillna(False, inplace=True)
/var/folders/b5/p9rmdc817y36m0hgtkqmx93m0000gn/T/ipykernel_89128/961496028.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

<b>Counter Attack</b>

In [36]:
for i in tqdm(range(len(dfShots_statsbomb_list))):    
    dfShots_statsbomb_list[i]['counter_attack'] = dfShots_statsbomb_list[i].play_pattern_name == 'From Counter'

100%|██████████████████████████████████████| 1321/1321 [00:01<00:00, 981.46it/s]


<b>preparing final set of coloumns to keep for learning</b>

In [37]:
# for i in range(len(dfShots_statsbomb_list)):
#     if "counter_attack" not in dfShots_statsbomb_list[i].columns:
#         print(i)
cols_to_keep = ['match_id', 'id', 'eventSec', 'period', 'goal', 'team_id', 'team_name',
                'player_id','shot_type_name', 'x', 'y', 'counter_attack',
                'fast_break', 'strong_foot', 'body_part_name', 'assist_type',
                'pass_end_x', 'pass_end_y', 'pass_switch', 'pass_cross',
                'pass_cut_back', 'pass_height_name', 'pass_technique_name',
                'carry_length', 'visible_angle', 'middle_angle', 'distance_to_goal', 
                'distance_visible_angle_interaction', 'log_distance', 'competition_gender',
                'shot_one_on_one', 'shot_open_goal', 'under_pressure']

In [38]:
for j in cols_to_keep:
    print("------------ ",j," ---------------\n")
    for i in range(len(dfShots_statsbomb_list)):
        if j not in dfShots_statsbomb_list[i].columns:
            print(i)

------------  match_id  ---------------

------------  id  ---------------

------------  eventSec  ---------------

------------  period  ---------------

------------  goal  ---------------

------------  team_id  ---------------

------------  team_name  ---------------

------------  player_id  ---------------

------------  shot_type_name  ---------------

------------  x  ---------------

------------  y  ---------------

------------  counter_attack  ---------------

------------  fast_break  ---------------

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146


In [39]:
for i in range(len(dfShots_statsbomb_list)):
    if "fast_break" not in dfShots_statsbomb_list[i].columns:
        dfShots_statsbomb_list[i]['fast_break'] = False

In [40]:
for i in tqdm(range(len(dfShots_statsbomb_list))):
    dfShots_statsbomb_list[i] = dfShots_statsbomb_list[i][cols_to_keep].copy()

100%|██████████████████████████████████████| 1321/1321 [00:05<00:00, 250.89it/s]


In [41]:
# no. of rows in the gk dataframe
total = 0
for i in range(len(df_gk_list)):
    total = total + df_gk_list[i].shape[0]
total

33627

In [42]:
# no. of rows in the shots dataframe
total = 0
for i in range(len(dfShots_statsbomb_list)):
    total = total + dfShots_statsbomb_list[i].shape[0]
total

33045

<b>finally merging the gk_player_id with shots dataframes</b>

In [43]:
for i in tqdm(range(len(dfShots_statsbomb_list))):
    dfShots_statsbomb_list[i] = dfShots_statsbomb_list[i].merge(
    df_gk_list[i][['id','player_id']],how='left',on='id',validate='m:1')

100%|██████████████████████████████████████| 1321/1321 [00:06<00:00, 207.91it/s]


In [44]:
for i in tqdm(range(len(dfShots_statsbomb_list))):
    dfShots_statsbomb_list[i].rename(columns={'player_id_y':'gk_player_id'},
                                     inplace=True)

100%|█████████████████████████████████████| 1321/1321 [00:00<00:00, 3272.30it/s]


<b>Adding some other stuff</b>

In [53]:
for i in tqdm(range(len(dfShots_statsbomb_list))):
    dfShots_statsbomb_list[i] = dfShots_statsbomb_list[i].loc[:,~dfShots_statsbomb_list[i].columns.duplicated()]

100%|██████████████████████████████████████| 1321/1321 [00:02<00:00, 440.51it/s]


In [80]:
for i in tqdm(range(len(dfShots_statsbomb_list))):
   dfShots_statsbomb_list[i] = dfShots_statsbomb_list[i].merge(
        events[i][['id','player_id','firstName','middleName', 'lastName', 'Name']],how='left')

100%|██████████████████████████████████████| 1321/1321 [00:11<00:00, 112.82it/s]


In [65]:
for i in tqdm(range(len(dfShots_statsbomb_list))):
    dfShots_statsbomb_list[i].rename(columns={'player_id_x':'player_id'},
                                     inplace=True)

100%|█████████████████████████████████████| 1321/1321 [00:00<00:00, 4516.08it/s]


<b>Checking for equal amount of column size</b>

In [84]:
for i in tqdm(range(len(dfShots_statsbomb_list))):
    if dfShots_statsbomb_list[i].shape[1]==38:
        continue
    else:
        print('oops!!!')

100%|███████████████████████████████████| 1321/1321 [00:00<00:00, 163649.34it/s]


<b>Finally concatenating the dataframes in the list to a single statsbomb shots data</b>

In [86]:
Statsbomb_Shots = pd.concat(dfShots_statsbomb_list)

/var/folders/b5/p9rmdc817y36m0hgtkqmx93m0000gn/T/ipykernel_89128/3313953556.py:1: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  Statsbomb_Shots = pd.concat(dfShots_statsbomb_list)


In [89]:
Statsbomb_Shots.to_csv('Statsbomb_Shots.csv')

In [98]:
!pip3 install shapely

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 11.9 MB/s eta 0:00:00a 0:00:01


In [99]:
from shapely.geometry import MultiPoint, Polygon, Point

In [101]:
!pip3 install geopandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 12.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 14.0 MB/s eta 0:00:0000:0100:01
  Using cached click-8.1.3-py3-none-any.whl (96 kB)
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
aws-sam-cli 1.51.0 requires click~=7.1, but you have click 8.1.3 which is incompatible.


In [102]:
import geopandas as gpd

<b>Filtering out penalty shots from freeze frames</b>

In [128]:
for i in tqdm(range(len(dfShots_statsbomb_list))):
    shots_id = dfShots_statsbomb_list[i].id
    freeze[i] = freeze[i][freeze[i].id.isin(shots_id)].copy()

100%|██████████████████████████████████████| 1321/1321 [00:11<00:00, 115.10it/s]


<b>Add Shot taker to the frame</b>

In [139]:
cols_to_keep = ['id', 'player_id', 'player_name', 'position_id', 'position_name', 'x', 'y', 'match_id']
df_shot_taker = []
for i in tqdm(range(len(freeze))):
    freeze_ids = freeze[i]['id'].unique()
    df_shot_taker.append(events[i].loc[events[i]['id'].isin(freeze_ids), cols_to_keep].copy())
    df_shot_taker[i]['teammate'] = True
    df_shot_taker[i]['event_freeze_id'] = 0
    freeze[i] = pd.concat([freeze[i], df_shot_taker[i]])

100%|███████████████████████████████████████| 1321/1321 [00:19<00:00, 67.35it/s]


In [140]:
for i in tqdm(range(len(freeze))):
    freeze[i].sort_values(['id', 'event_freeze_id'], inplace=True)
    freeze[i].drop_duplicates(['id', 'player_id'], keep='first', inplace=True)

100%|██████████████████████████████████████| 1321/1321 [00:05<00:00, 240.83it/s]


In [143]:
dfFreeze = pd.concat(freeze)

<b>Calculating Number of Opposition Players while the shot was being taken</b>

In [151]:
from mplsoccer import Pitch, Standardizer
statsbomb_pitch = Pitch()
shot_takers = dfFreeze[dfFreeze['event_freeze_id'] == 0].copy()  # we added this earlier
shot_takers.reset_index(drop=True, inplace=True)

verts = np.zeros((len(shot_takers), 3, 2))
verts[:, 1:, :] = statsbomb_pitch.goal_right
verts[:, 0, :] = shot_takers[['x', 'y']].values
verts = gpd.GeoSeries([Polygon(vert).buffer(0) for vert in verts])  # the angle to the goal polygon, buffer added as sometimes shot is on the goal line
verts = gpd.GeoDataFrame({'id': shot_takers['id'], 'goal_angle': gpd.GeoSeries(verts)})

player_positions = gpd.GeoSeries.from_xy(dfFreeze['x'], dfFreeze['y'])
player_positions = gpd.GeoDataFrame({'id': dfFreeze['id'], 'event_freeze_id': dfFreeze['event_freeze_id'], 'position': player_positions})

# merge the goal angle onto the player positions
player_positions = gpd.GeoDataFrame(player_positions.merge(verts, on='id'))

# add a mask for whether the player intersects with the goal angle
player_positions['n_angle'] = player_positions['position'].intersects(player_positions['goal_angle'])
dfFreeze = dfFreeze.merge(player_positions[['id', 'event_freeze_id', 'n_angle']], on=['id', 'event_freeze_id'])

# calculate the number of defenders in the goal angle
num_in_goal_angle = dfFreeze[dfFreeze['teammate'] == False].groupby('id')['n_angle'].sum().reset_index()

In [152]:
standard = Standardizer(pitch_from='statsbomb', pitch_to='uefa')
x_std, y_std = standard.transform(dfFreeze['x'], dfFreeze['y'])
dfFreeze['x'] = x_std
dfFreeze['y'] = y_std

<b>Calculating the area_goal and area_shot features</b>

In [153]:
area_goal = []
area_shot = []

for shot_id in tqdm(dfFreeze['id'].unique()):
    subset = dfFreeze.loc[dfFreeze['id'] == shot_id,
                                     ['x', 'y', 'teammate', 'event_freeze_id', 'position_id','position_name']].copy()
    team1, team2 = statsbomb_pitch.voronoi(subset['x'], subset['y'], subset['teammate'])
    subset['rank'] = subset.groupby('teammate')['x'].cumcount()

    gk_rank = subset.loc[(subset['teammate'] == False) & (subset['position_id'] == 1), 'rank']
    if len(gk_rank) > 0:
        area_goal.append(Polygon(team2[gk_rank.iloc[0]]).area)
    else:
        area_goal.append(0)

    shot_taker_voronoi = team1[subset.loc[subset['event_freeze_id'] == 0, 'rank'].iloc[0]]
    area_shot.append(Polygon(shot_taker_voronoi).area)

df_freeze_features = pd.DataFrame({'id': dfFreeze['id'].unique(),
                                   'area_shot': area_shot,
                                   'area_goal': area_goal}
                                 )

100%|███████████████████████████████████| 33045/33045 [1:05:08<00:00,  8.46it/s]


In [155]:
gk_position = dfFreeze.loc[(dfFreeze['position_name'] == 'Goalkeeper') &
                                      (dfFreeze['teammate'] == False),
                                      ['id', 'x', 'y']]
gk_position.rename({'x': 'goalkeeper_x',
                    'y': 'goalkeeper_y'},
                   axis=1, inplace=True)

In [156]:
df_freeze_features = (df_freeze_features
                      .merge(num_in_goal_angle, how='left', on='id', validate='1:1')
                      .merge(gk_position, how='left', on='id', validate='1:1')
                     )

In [158]:
df_freeze_features.rename(columns={'n_angle':'opps_in_n_angle'},inplace=True)

In [160]:
df_freeze_features.to_csv('Statsbomb_Shots_FreezeFeatures.csv')

In [169]:
Statsbomb_Shots = Statsbomb_Shots.merge(df_freeze_features, on='id', how='left', validate='1:1')

In [170]:
Statsbomb_Shots.loc[(Statsbomb_Shots['pass_technique_name'].isnull()) &
                   (Statsbomb_Shots['assist_type']=='pass'), 'pass_technique_name'] = 'other'

<b> Shots taken, Goals, Goals Conversion percentage info</b>

In [175]:
print('Number of Shots taken in the dataset : ', Statsbomb_Shots.shape[0])
print('Number of Goals in the dataset : ', Statsbomb_Shots.goal.value_counts()[1])
print('Goals conversion percentage : ', Statsbomb_Shots.goal.value_counts()[1]/Statsbomb_Shots.shape[0])

Number of Shots taken in the dataset :  33045
Number of Goals in the dataset :  3710
Goals conversion percentage :  0.11227114540777727
